In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
def convert_df(file_path, task):
    with open(file_path, encoding='utf-8') as fr:
        lines = fr.readlines()
    label_list = []
    sentence_list = []
    ids = []
    for line in lines:
        json_str = json.loads(line)
        if task == 'test':
            ids.append(json_str['id'])
            sentence_list.append(json_str['sentence'])
        else:
            label_list.append(json_str['label'])
            sentence_list.append(json_str['sentence'])
    if task == 'test':
        data_dict = {'id': ids, 'text': sentence_list}
    else:
        data_dict = {'label': label_list, 'text': sentence_list}
    data = pd.DataFrame(data_dict)
    
    return data

In [3]:
train_path = './data/train.json'
dev_path = './data/dev.json'
test_path = './data/test.json'

train_data = convert_df(train_path, 'train')
print(train_data.head(5))

dev_data = convert_df(dev_path, 'dev')
print(dev_data.head(5))

  label                                            text
0   108    上课时学生手机响个不停，老师一怒之下把手机摔了，家长拿发票让老师赔，大家怎么看待这种事？
1   104  商赢环球股份有限公司关于延期回复上海证券交易所对公司2017年年度报告的事后审核问询函的公告
2   106                通过中介公司买了二手房，首付都付了，现在卖家不想卖了。怎么处理？
3   112                             2018年去俄罗斯看世界杯得花多少钱？
4   109                           剃须刀的个性革新，雷明登天猫定制版新品首发
  label                            text
0   102      江疏影甜甜圈自拍，迷之角度竟这么好看，美吸引一切事物
1   110  以色列大规模空袭开始！伊朗多个军事目标遭遇打击，誓言对等反击
2   104           出栏一头猪亏损300元，究竟谁能笑到最后！
3   109                以前很火的巴铁为何现在只字不提？
4   112   作为一名酒店从业人员，你经历过房客哪些特别没有素质的行为？


In [4]:
train_data['word_cnt'] = train_data['text'].apply(lambda x: len(x))
print(train_data.head(5))

  label                                            text  word_cnt
0   108    上课时学生手机响个不停，老师一怒之下把手机摔了，家长拿发票让老师赔，大家怎么看待这种事？        44
1   104  商赢环球股份有限公司关于延期回复上海证券交易所对公司2017年年度报告的事后审核问询函的公告        46
2   106                通过中介公司买了二手房，首付都付了，现在卖家不想卖了。怎么处理？        32
3   112                             2018年去俄罗斯看世界杯得花多少钱？        19
4   109                           剃须刀的个性革新，雷明登天猫定制版新品首发        21


In [5]:
train_data['word_cnt'].describe()

count    53360.000000
mean        22.131241
std          7.309860
min          2.000000
25%         17.000000
50%         22.000000
75%         28.000000
max        145.000000
Name: word_cnt, dtype: float64

In [6]:
train_data['label'].value_counts()

109    5955
104    5200
102    4976
113    4851
107    4118
101    4081
103    3991
110    3632
108    3437
116    3390
112    3368
115    2886
106    2107
100    1111
114     257
Name: label, dtype: int64

In [16]:
# 分词去掉一些无用词
import jieba
def cut_with_jieba(text, filter=None):
    if filter:
        for c in filter:
            text = text.replace(c, '')
    words = ['Number' if word.isdigit() else word for word in jieba.cut(text)]
    # todo 停用词表还可以加进来
    return ' '.join(words)

filter = './?？；。（()）【】{}[]!！，,<>《》+'

train_data['words'] = train_data['text'].apply(lambda x: cut_with_jieba(x, filter))

print(train_data.head(5))

  label                                            text  word_cnt  \
0   108    上课时学生手机响个不停，老师一怒之下把手机摔了，家长拿发票让老师赔，大家怎么看待这种事？        44   
1   104  商赢环球股份有限公司关于延期回复上海证券交易所对公司2017年年度报告的事后审核问询函的公告        46   
2   106                通过中介公司买了二手房，首付都付了，现在卖家不想卖了。怎么处理？        32   
3   112                             2018年去俄罗斯看世界杯得花多少钱？        19   
4   109                           剃须刀的个性革新，雷明登天猫定制版新品首发        21   

                                               words  
0  上课时 学生 手机 响个 不停 老师 一怒之下 把 手机 摔 了 家长 拿 发票 让 老师 ...  
1  商赢 环球 股份 有限公司 关于 延期 回复 上海证券交易所 对 公司 Number 年 年...  
2       通过 中介 公司 买 了 二手房 首付 都 付 了 现在 卖家 不想 卖 了 怎么 处理  
3                       Number 年 去 俄罗斯 看 世界杯 得花 多少 钱  
4                      剃须刀 的 个性 革新 雷明登 天猫 定制 版 新品 首发  


In [17]:
dev_data['words'] = dev_data['text'].apply(lambda x: cut_with_jieba(x, filter))
print(dev_data.head(5))

  label                            text  \
0   102      江疏影甜甜圈自拍，迷之角度竟这么好看，美吸引一切事物   
1   110  以色列大规模空袭开始！伊朗多个军事目标遭遇打击，誓言对等反击   
2   104           出栏一头猪亏损300元，究竟谁能笑到最后！   
3   109                以前很火的巴铁为何现在只字不提？   
4   112   作为一名酒店从业人员，你经历过房客哪些特别没有素质的行为？   

                                       words  
0      江 疏影 甜甜 圈自 拍迷 之 角度 竟 这么 好看 美 吸引 一切 事物  
1  以色列 大规模 空袭 开始 伊朗 多个 军事 目标 遭遇 打击 誓言 对 等 反击  
2          出栏 一头 猪 亏损 Number 元 究竟 谁 能 笑 到 最后  
3                      以前 很火 的 巴铁 为何 现在 只字不提  
4   作为 一名 酒店 从业人员 你 经历 过 房客 哪些 特别 没有 素质 的 行为  


In [18]:
# 写入到文件
train_data[['words', 'label']].to_csv('./data/train_data.csv', sep='\t', encoding='utf-8', index=None)
dev_data[['words', 'label']].to_csv('./data/dev_data.csv', sep='\t', encoding='utf-8', index=None)

In [19]:

# 准备测试集
test_data = convert_df(test_path, 'test')

test_data['words'] = test_data['text'].apply(lambda x: cut_with_jieba(x, filter))

print(test_data.head(5))

test_data[['id', 'words']].to_csv('./data/test_data.csv', sep='\t', encoding='utf-8', index=None)


   id                            text  \
0   0  在设计史上，每当相对稳定的发展时期，这种设计思想就会成为主导   
1   1          利希施泰纳宣布赛季结束后离队：我需要新的挑战   
2   2                   庄家一般都是什么操盘思路？   
3   3                  王者荣耀里搅屎棍英雄都有谁？   
4   4          照片不小心被删，看看下面的教程，完美找回来！   

                                         words  
0  在 设计 史上 每当 相对 稳定 的 发展 时期 这种 设计 思想 就 会 成为 主导  
1           利希 施泰纳 宣布 赛季 结束 后 离队 ： 我 需要 新 的 挑战  
2                           庄家 一般 都 是 什么 操盘 思路  
3                       王者 荣耀 里 搅 屎 棍 英雄 都 有 谁  
4                照片 不 小心 被删 看看 下面 的 教程 完美 找 回来  
